In [7]:
!pip install dotenv

Defaulting to user installation because normal site-packages is not writeable


In [13]:
!pip install requests

Defaulting to user installation because normal site-packages is not writeable


In [26]:
import os
from dotenv import load_dotenv
import requests
import json
from pprint import pprint

load_dotenv()

api_key = os.getenv("SERP_API_KEY")
if not api_key:
    raise ValueError("Missing SERP_API_KEY environment variable")   

Now we create a query and then use the google_pattent base url form serpApi to get the patent data on lithium battery

In [9]:
query = "lithium battery"

url = f"https://serpapi.com/search?engine=google_patents&q={query}&api_key={api_key}"

In [21]:
response = requests.get(url)

if response.status_code == 200:
    data = response.json()
    
    patents = data.get("organic_results",[])

    with open("files/patents.json", "w") as f:
        json.dump(patents, f,indent=4)
    
    # indent=4: This is an optional argument that makes the JSON file human-readable. It formats the file with an indentation of 4 spaces, 
    # creating a "pretty-printed" output. Without this, the entire JSON would be written on a single line.

here the extracted jason file is stored in hte patents.jason file in our files folder

In [ ]:
# see this is the url of the first element of the patent.jason file 
patent_url_1 = f"https://serpapi.com/search.json?engine=google_patents_details&patent_id=patent%2FCN109659487B%2Fen&api_key={api_key}"

if response.status_code == 200:
    response = requests.get(patent_url_1)
    data = response.json()
    
    with open("files/patent_details.json", "w") as f:
        json.dump(data, f,indent=4) 

now as the file is very big and also consists of a lot of metadata and other data which is currently not required, as currently we are only considered with the text data.

so we do the serpapi link of the first element of our jason file and extract it in a new file

In [23]:
data.keys()

dict_keys(['search_metadata', 'search_parameters', 'title', 'type', 'pdf', 'publication_number', 'country', 'prior_art_keywords', 'prior_art_date', 'application_number', 'inventors', 'assignees', 'priority_date', 'filing_date', 'publication_date', 'worldwide_applications', 'events', 'external_links', 'images', 'classifications', 'abstract', 'abstract_original', 'description_link', 'claims', 'priority_applications', 'applications_claiming_priority', 'family_id', 'patent_citations', 'non_patent_citations', 'cited_by', 'similar_documents', 'legal_events', 'concepts'])

In [25]:
data.get('abstract',"")

'The invention belongs to the field of lithium batteries, and discloses a pre-lithiation method for lithium metal anode protection, which is characterized by comprising the following steps: (1) coating the surface of a current collector to form an organic polymer film; (2) coating The current collector covered with the organic polymer film is assembled into a battery together with the lithium sheet, the diaphragm and the electrolyte for discharge treatment, and metal lithium is deposited on the surface of the current collector; (3) The current collector with the metal lithium deposited on the surface is taken out, that is, to obtain A negative electrode material with a protective film on the surface that can be used in lithium metal batteries. The metal lithium negative electrode prepared by the pre-lithiation method of the present invention can suppress the generation of lithium dendrites on the one hand, and on the other hand, the protective film formed in situ can prevent the reacti

In [27]:
pprint(data.get('abstract',""))

('The invention belongs to the field of lithium batteries, and discloses a '
 'pre-lithiation method for lithium metal anode protection, which is '
 'characterized by comprising the following steps: (1) coating the surface of '
 'a current collector to form an organic polymer film; (2) coating The current '
 'collector covered with the organic polymer film is assembled into a battery '
 'together with the lithium sheet, the diaphragm and the electrolyte for '
 'discharge treatment, and metal lithium is deposited on the surface of the '
 'current collector; (3) The current collector with the metal lithium '
 'deposited on the surface is taken out, that is, to obtain A negative '
 'electrode material with a protective film on the surface that can be used in '
 'lithium metal batteries. The metal lithium negative electrode prepared by '
 'the pre-lithiation method of the present invention can suppress the '
 'generation of lithium dendrites on the one hand, and on the other hand, the '
 '

Here we can see how the abstract of the file is useful to us and we need it so now we write the code "information_collector.py and helper.py" in which we iterate through the entire jason file and extract that abstact part out of each element of the jason file

# we created the helper.py and information_collector.py file and with the help of it in the resuls folder we have our .jason file available
# now we simple extract the "abstract" content file form it using this code below

In [29]:
dir_files = os.listdir("results")

for file in dir_files:
    if file.endswith(".json"):
        with open(f"results/{file}", "r") as f:
            data = json.load(f)
            pprint(data.get('abstract',""))
            print("\n"+ "="*40 + "\n")

('The invention relates to a lithium negative electrode with a functional '
 'protective layer and a lithium-sulfur battery. The lithium negative '
 'electrode has a functional protective layer coated on the surface and '
 'contains a conductive polymer. In the lithium-sulfur battery, the functional '
 'protective layer of the lithium negative electrode is in contact with the '
 'electrolyte, so that a stable interface is formed between the electrolyte '
 'and the matrix of the lithium negative electrode.')


('The invention discloses a composite separator for lithium batteries and its '
 'preparation method and application. The composite separator comprises a '
 'polymer base film, a vulcanized polyacrylonitrile ion-conductive particle '
 'coating on one side of the polymer base film. layer and an inorganic ceramic '
 'particle coating and/or polymer modification layer on the other side of the '
 'polymer substrate film. The polymer-ceramic composite separator can be used '
 'alone or

# now we need to use the embedding model to create the embedding

In [30]:
emd_url = "http://localhost:11434/api/embeddings"

headers = {
    "Content-Type": "application/json"
}   

data = {
    "model": "nomic-embed-text",
    "prompt": "The sky is blue because of Rayleigh scattering" 
}

response = requests.post(emd_url, headers=headers, json=data)

response_data = response.json()
if response.status_code == 200:
    embedding = response_data.get("embedding", [])
    print(f"Embedding length: {len(embedding)}")
    print(embedding)

Embedding length: 768
[0.5897541046142578, 0.4019525945186615, -3.305915355682373, -0.5254098176956177, 0.7496400475502014, 1.5190777778625488, -0.12491098046302795, 0.39693623781204224, 0.06786676496267319, -1.108647108078003, 0.6920070648193359, 1.278577446937561, 1.1457475423812866, 1.088091254234314, 0.2514374852180481, 0.29303377866744995, 0.1521180123090744, -0.633834183216095, -0.21033339202404022, -0.19534991681575775, -1.7955280542373657, -0.6291974782943726, 0.037852637469768524, -0.6675937175750732, 1.2617499828338623, 1.2767579555511475, -0.1600552499294281, -0.001568942447192967, -0.29807400703430176, -0.48047590255737305, 1.2062546014785767, -0.6384469866752625, -0.5399971008300781, -1.0354348421096802, 0.6303300857543945, -1.208932876586914, 0.6832435727119446, -0.05861659720540047, -0.19714537262916565, 0.12849006056785583, -0.014737173914909363, -0.5534937977790833, 0.351531058549881, 0.044485289603471756, 0.5978081822395325, -0.9545161724090576, 0.5082172155380249, 1.

# so see now we have a demo fuction ready for the embeddings we use this only to make our embedding.py file

# now we also created a docker-compose.yml file as this is used in order to download the vector database on our system in one go

we are using the opensearch vector database